# Orders API & Planet SDK 

Getting started with Planet SDK and the Orders API.

## Authentication

The new SDK supports a couple different methods of authentication (and soon, will support OIDC / Okta token-based authentication as well). [Read more in the docs here](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/python/sdk-guide/).

For this example, I'll use the `getpass` tool to pass in my API key as a string to the `from_key` method.

In [ ]:
from getpass import getpass
from planet import Auth
api_key = getpass('Enter your API key:')

auth = Auth.from_key(api_key)

# Demo

## Building an Order request

### Method 1: Old style

With this method, you can build an Order request by directly using a blob of JSON -- similar to how you might have built an Order in the Python Client V1, or directly using the API via e.g., `requests`:

In [ ]:
request = {
  "name": "test_order_sdk_method_1",
  "products": [
    {
     "item_ids":[  
        "20200922_183724_23_106a",
        "20200922_183722_17_106a"
         ],
      "item_type": "PSScene",
      "product_bundle": "analytic_udm2"
    }
  ],
   "tools": [
    {
      "reproject": {
        "projection": "EPSG:4326",
        "kernel": "cubic"
      }
    }
  ]
}

### Method 2: New style

In V2 ofthe SDK, you can also build an Order request object using the new `order_request` functionality -- here you can specify all Order details including products, bundles & fallback bundles, cloud delivery configuration, tools & toolchain operations, etc. [Read more in the docs here](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/python/sdk-guide/).

In [ ]:
from planet import order_request

item_ids = ["20200922_183724_23_106a", "20200922_183722_17_106a"]

products = [
    order_request.product(item_ids, 'analytic_udm2', 'PSScene')
]

tools = [
    order_request.reproject_tool(projection='EPSG:4326', kernel='cubic')
]

request = order_request.build_request(
    'test_order_sdk_method_2', products=products, tools=tools)


## Create the Order

Regardless of the method you use to build your order, the next step after building an Order is to send a "create" request to the Orders API.

To do this, we'll create a `Session` to manage our communcation with Planet in general -- this will make use of that `auth` object we created earlier. Within the context of this Session, we'll create an Orders API-specific `client` to handle interactions with the Orders API.

In [ ]:
from planet import Session, OrdersClient

# an async Orders client to request order creation
async def main():
  async with Session(auth=auth) as sess:
    cl = OrdersClient(sess)
    order = await cl.create_order(request)

# async magic to remember: "async def" to create a coroutine, then "await" to make it run
await main()

## Alternative: Create the Order, then monitor status until complete

In [ ]:
from planet import reporting, Session, OrdersClient

# remember: "async def" to create the async coroutine
async def create_poll_and_download():
    async with Session(auth=auth) as sess:
        cl = OrdersClient(sess)

        # Use "reporting" to manage polling for order status
        with reporting.StateBar(state='creating') as bar:
            # create order via Orders client
            order = await cl.create_order(request)
            bar.update(state='created', order_id=order['id'])

            # poll...poll...poll...
            await cl.wait(order['id'], callback=bar.update_state)

        # if we get here that means the order completed. Yay! Download the files.
        await cl.download_order(order['id'])

# remember: "await" to run the thing
await create_poll_and_download()